# Exploración - Step 2: Transformación

In [1]:
import os
from math import cos, radians, sin
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
os.chdir("..")
os.getcwd()

'/home/troonies/repos/itba-aws-mle'

## Data

Detailed info on **flight arrivals and delays for U.S. airports**, categorized by carriers.

It includes metrics such as the number of arriving flights, delays over 15 minutes, cancellation and diversion counts, and the breakdown of delays attributed to carriers, weather, NAS (National Airspace System), security, and late aircraft arrivals.

In [3]:
cols_descriptions = {
    "Date":          "The date of observation",
    "Location":      "The common name of the location of the weather station",
    "MinTemp":       "The minimum temperature in degrees celsius",
    "MaxTemp":       "The maximum temperature in degrees celsius",
    "Rainfall":      "The amount of rainfall recorded for the day in mm",
    "Evaporation":   "The so-called Class A pan evaporation (mm) in the 24 hours to 9am",
    "Sunshine":      "The number of hours of bright sunshine in the day.",
    "WindGustDir":   "The direction of the strongest wind gust in the 24 hours to midnight",
    "WindGustSpeed": "The speed (km/h) of the strongest wind gust in the 24 hours to midnight",
    "WindDir9am":    "Direction of the wind at 9am",
    "WindDir3pm":    "Direction of the wind at 3pm",
    "WindSpeed9am":  "Wind speed (km/hr) averaged over 10 minutes prior to 9am",
    "WindSpeed3pm":  "Wind speed (km/hr) averaged over 10 minutes prior to 3pm",
    "Humidity9am":   "Humidity (percent) at 9am",
    "Humidity3pm":   "Humidity (percent) at 3pm",
    "Pressure9am":   "Atmospheric pressure (hpa) reduced to mean sea level at 9am",
    "Pressure3pm":   "Atmospheric pressure (hpa) reduced to mean sea level at 3pm",
    "Cloud9am":      "Fraction of sky obscured by cloud at 9am. This is measured in 'oktas', "
                     "which are a unit of eigths. It records how many eigths of the sky are obscured by cloud. "
                     "A 0 measure indicates completely clear sky whilst an 8 indicates that it is completely overcast.",
    "Cloud3pm":      "Fraction of sky obscured by cloud (in 'oktas': eighths) at 3pm. See Cload9am for a description of the values",
    "Temp9am":       "Temperature (degrees C) at 9am",
    "Temp3pm":       "Temperature (degrees C) at 3pm",
    "RainToday":     "Boolean: 1 if precipitation (mm) in the 24 hours to 9am exceeds 1mm, otherwise 0",
    "RainTomorrow":  "The amount of next day rain in mm. Used to create response variable RainTomorrow. A kind of measure of the 'risk'",
}

In [4]:
cols_types = {
    "int": [
        "WindGustSpeed",
        "WindSpeed9am",
        "WindSpeed3pm",
        "Humidity9am",
        "Humidity3pm",
        "Cloud9am",
        "Cloud3pm",
    ],
    "float": [
        "MinTemp",
        "MaxTemp",
        "Rainfall",
        "Evaporation",
        "Sunshine",
        "WindGustDir",
        "WindDir9am",
        "WindDir3pm",
        "Pressure9am",
        "Pressure3pm",
        "Temp9am",
        "Temp3pm",
        "WindGustDir_east",  # engineered feature
        "WindGustDir_north",  # engineered feature
        "WindDir9am_east",  # engineered feature
        "WindDir9am_north",  # engineered feature
        "WindDir3pm_east",  # engineered feature
        "WindDir3pm_north",  # engineered feature
    ],
    "bool": [
        "RainToday",
    ],
    "cat": [
        "Location",
    ],
}
# ? "RainTomorrow" is not included because it's the target

assert sum(len(cts) for cts in cols_types.values()) == len(set(cols_types["int"]) | set(cols_types["float"]) | set(cols_types["bool"]) | set(cols_types["cat"]))

In [5]:
SELECTED_COLS = [
    "Date",
    # "Location",  # ! not needed for this use case
    "MinTemp",
    "MaxTemp",
    "Rainfall",
    # "Evaporation",  # ! too many nulls
    # "Sunshine",  # ! too many nulls
    "WindGustDir",  # ? will be transformed
    "WindGustSpeed",
    "WindDir9am",  # ? will be transformed
    "WindDir3pm",  # ? will be transformed
    "WindSpeed9am",
    "WindSpeed3pm",
    "Humidity9am",
    "Humidity3pm",
    "Pressure9am",
    "Pressure3pm",
    # "Cloud9am",  # ! too many nulls
    # "Cloud3pm",  # ! too many nulls
    "Temp9am",
    "Temp3pm",
    "RainToday",  # ? will be transformed
    "RainTomorrow",  # * target variable (will be transformed)
]

Use training and validation splits only for the EDA.

In [6]:
df = pd.read_csv("data/train_val.csv", usecols=SELECTED_COLS)
df

,Date,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2012-01-26,20.0,31.6,0.4,ESE,39.0,E,ENE,17.0,9.0,69.0,55.0,1007.9,1005.5,25.2,29.9,No,No
1,2010-07-12,11.2,18.2,0.0,WNW,28.0,SW,SE,7.0,11.0,58.0,62.0,1023.3,1020.7,14.3,17.3,No,No
2,2015-06-08,11.8,24.5,0.2,ENE,15.0,SW,NE,6.0,6.0,71.0,56.0,1028.0,1024.2,17.1,21.1,No,No
3,2012-04-05,14.6,19.1,9.8,SSW,50.0,SW,S,22.0,24.0,1.0,1.0,1020.9,1023.2,17.9,16.8,Yes,No
4,2016-03-05,15.4,19.5,3.8,ESE,30.0,SE,SSE,13.0,17.0,97.0,80.0,1021.6,1020.1,16.9,18.2,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127968,2009-01-18,10.1,29.7,0.0,ENE,33.0,WSW,NW,4.0,13.0,63.0,26.0,1020.2,1016.1,14.4,26.8,No,No
127969,2016-09-28,6.2,12.9,14.0,SSW,59.0,S,S,17.0,15.0,75.0,71.0,1021.5,1023.1,10.2,10.6,Yes,No
127970,2016-11-20,18.0,30.0,1.8,NE,28.0,N,ENE,9.0,13.0,83.0,47.0,1021.5,1017.0,19.9,28.8,Yes,No
127971,2012-11-04,9.5,32.9,0.0,NW,46.0,E,N,2.0,20.0,84.0,24.0,1013.0,1007.2,15.6,31.2,No,No


## Transformations

In [7]:
df.head()

,Date,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2012-01-26,20.0,31.6,0.4,ESE,39.0,E,ENE,17.0,9.0,69.0,55.0,1007.9,1005.5,25.2,29.9,No,No
1,2010-07-12,11.2,18.2,0.0,WNW,28.0,SW,SE,7.0,11.0,58.0,62.0,1023.3,1020.7,14.3,17.3,No,No
2,2015-06-08,11.8,24.5,0.2,ENE,15.0,SW,NE,6.0,6.0,71.0,56.0,1028.0,1024.2,17.1,21.1,No,No
3,2012-04-05,14.6,19.1,9.8,SSW,50.0,SW,S,22.0,24.0,1.0,1.0,1020.9,1023.2,17.9,16.8,Yes,No
4,2016-03-05,15.4,19.5,3.8,ESE,30.0,SE,SSE,13.0,17.0,97.0,80.0,1021.6,1020.1,16.9,18.2,Yes,No


### Functions

In [8]:
def cast_everything_to_float(df: pd.DataFrame) -> pd.DataFrame:
    """Cast all numeric columns to float."""
    df = df.astype({c: float for c in cols_types["int"] if c in df.columns})
    df = df.astype({c: float for c in cols_types["float"] if c in df.columns})
    df = df.astype({c: float for c in cols_types["bool"] if c in df.columns})
    return df

def drop_null_rows(df: pd.DataFrame, aceptable_nulls: int = 2) -> pd.DataFrame:
    """Drop rows that have too many nulls."""
    return df[df.isnull().sum(axis=1) <= aceptable_nulls].reset_index(drop=True)

def standardize_cols(df: pd.DataFrame, df_desc: pd.DataFrame) -> pd.DataFrame:
    """Standardize columns."""
    for c in (cols_types["int"] + cols_types["float"] + cols_types["bool"]):
        if c in df.columns:
            df[c] = df[c] - df_desc.at["mean", c]
            df[c] = df[c] / df_desc.at["std", c]
    return df

def limit_outliers(df: pd.DataFrame, df_whiskers: pd.DataFrame) -> pd.DataFrame:
    """Limit outlier values by rounding them to the corresponding boxplot whisker value."""
    for c in (cols_types["int"] + cols_types["float"]):  # boolean is not necessary
        if c not in df.columns:
            continue

        low_w = df_whiskers.at[c, "lower_whisker"]
        high_w = df_whiskers.at[c, "upper_whisker"]

        df[f"{c}_lower_out"] = df[c] < low_w
        df[f"{c}_high_out"] = high_w < df[c]

        if df[f"{c}_lower_out"].any():
            df[c] = df[[c, f"{c}_lower_out"]].apply(lambda row: low_w if row[f"{c}_lower_out"] else row[c], axis=1)
        del df[f"{c}_lower_out"], low_w

        if df[f"{c}_high_out"].any():
            df[c] = df[[c, f"{c}_high_out"]].apply(lambda row: high_w if row[f"{c}_high_out"] else row[c], axis=1)
        del df[f"{c}_high_out"], high_w

    return df

In [9]:
ANGLE_STEP = 22.5
wind_dir_to_radians = {
    "E":   radians(0.0),
    "ENE": radians(ANGLE_STEP),
    "NE":  radians(2.0 * ANGLE_STEP),
    "NNE": radians(3.0 * ANGLE_STEP),
    "N":   radians(4.0 * ANGLE_STEP),
    "NNW": radians(5.0 * ANGLE_STEP),
    "NW":  radians(6.0 * ANGLE_STEP),
    "WNW": radians(7.0 * ANGLE_STEP),
    "W":   radians(8.0 * ANGLE_STEP),
    "WSW": radians(9.0 * ANGLE_STEP),
    "SW":  radians(10.0 * ANGLE_STEP),
    "SSW": radians(11.0 * ANGLE_STEP),
    "S":   radians(12.0 * ANGLE_STEP),
    "SSE": radians(13.0 * ANGLE_STEP),
    "SE":  radians(14.0 * ANGLE_STEP),
    "ESE": radians(15.0 * ANGLE_STEP),
}
wind_dir_to_radians

{'E': 0.0,
 'ENE': 0.39269908169872414,
 'NE': 0.7853981633974483,
 'NNE': 1.1780972450961724,
 'N': 1.5707963267948966,
 'NNW': 1.9634954084936207,
 'NW': 2.356194490192345,
 'WNW': 2.748893571891069,
 'W': 3.141592653589793,
 'WSW': 3.5342917352885173,
 'SW': 3.9269908169872414,
 'SSW': 4.319689898685966,
 'S': 4.71238898038469,
 'SSE': 5.105088062083414,
 'SE': 5.497787143782138,
 'ESE': 5.8904862254808625}

In [11]:
def fill_nulls(df: pd.DataFrame, df_desc: pd.DataFrame) -> pd.DataFrame:
    """Fill nulls programatically."""
    null_counts = df.isnull().sum()

    for c in (cols_types["int"] + cols_types["float"]):
        if (c not in df.columns) or (null_counts.at[c] == 0):
            continue
        df[c] = df[c].fillna(df_desc.at["mean", c])

    for c in cols_types["bool"]:
        if (c not in df.columns) or (null_counts.at[c] == 0):
            continue

        # We fill with the most repeated boolean value (most likely a std float by now)
        col_vc = df[c].value_counts()
        df[c] = df[c].fillna(col_vc.iat[0])

    return df

### Apply transformations

In [12]:
df_new = df.copy()

In [13]:
df_new["Date"] = pd.to_datetime(df_new["Date"])

df_new["RainToday"] = df_new["RainToday"].map(lambda rt: np.nan if pd.isna(rt) else (rt == "Yes"))
df_new["RainTomorrow"] = df_new["RainTomorrow"] == "Yes"
df_new = df_new[["RainTomorrow", "Date"] + [c for c in df_new.columns if c not in ["RainTomorrow", "Date"]]]

wind_cols = ["WindGustDir", "WindDir9am", "WindDir3pm"]
for c in wind_cols:
    df_new[f"{c}_rad"] = df_new[c].map(wind_dir_to_radians)
    df_new[f"{c}_east"] = df_new[f"{c}_rad"].map(lambda r: cos(r))
    df_new[f"{c}_north"] = df_new[f"{c}_rad"].map(lambda r: sin(r))
    df_new = df_new.drop([c, f"{c}_rad"], axis=1)

In [14]:
df_new = cast_everything_to_float(df_new)
df_new = drop_null_rows(df_new)

In [15]:
df_desc_orig = df_new.describe()
df_desc_orig

,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,WindGustDir_east,WindGustDir_north,WindDir9am_east,WindDir9am_north,WindDir3pm_east,WindDir3pm_north
count,117590,117542.000000,117574.000000,116795.000000,117590.000000,117590.000000,117590.000000,116946.000000,116389.000000,109633.000000,109681.000000,117515.000000,117005.000000,116795.000000,1.175790e+05,117579.000000,1.121200e+05,112120.000000,1.172320e+05,117232.000000
mean,2013-04-16 16:35:27.609490944,12.192230,23.352066,2.305087,40.086232,14.300272,18.843728,68.451482,50.984414,1017.621875,1015.212410,17.030144,21.786366,0.220549,-1.953347e-02,-0.050254,2.703677e-02,0.009904,-2.380697e-02,-0.042755
min,2007-11-01 00:00:00,-8.500000,-4.800000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,982.000000,977.100000,-7.200000,-5.400000,0.000000,-1.000000e+00,-1.000000,-1.000000e+00,-1.000000,-1.000000e+00,-1.000000
25%,2011-01-31 00:00:00,7.500000,17.900000,0.000000,31.000000,7.000000,13.000000,56.000000,36.000000,1012.900000,1010.400000,12.200000,16.600000,0.000000,-7.071068e-01,-0.707107,-7.071068e-01,-0.707107,-7.071068e-01,-0.707107
50%,2013-06-16 00:00:00,12.000000,22.800000,0.000000,39.000000,13.000000,19.000000,69.000000,51.000000,1017.600000,1015.200000,16.700000,21.300000,0.000000,-1.836970e-16,0.000000,6.123234e-17,0.000000,-1.836970e-16,0.000000
75%,2015-06-21 00:00:00,16.900000,28.500000,0.600000,48.000000,20.000000,24.000000,83.000000,65.000000,1022.400000,1020.000000,21.700000,26.600000,0.000000,7.071068e-01,0.707107,7.071068e-01,0.707107,7.071068e-01,0.707107
max,2017-06-25 00:00:00,33.900000,48.100000,367.600000,135.000000,87.000000,87.000000,100.000000,100.000000,1041.000000,1039.600000,40.200000,46.700000,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000,1.000000e+00,1.000000
std,NaN,6.445237,7.155305,8.318535,13.490377,8.715115,8.620551,19.206530,20.838074,7.089881,7.020179,6.550717,6.981686,0.414619,7.120533e-01,0.700058,6.922264e-01,0.721112,7.100090e-01,0.702496


In [16]:
df_new = standardize_cols(df_new, df_desc=df_desc_orig)

In [17]:
df_new.head()

,RainTomorrow,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,WindGustDir_east,WindGustDir_north,WindDir9am_east,WindDir9am_north,WindDir3pm_east,WindDir3pm_north
0,False,2012-01-26,1.211401,1.152702,-0.229017,-0.080519,0.309775,-1.141891,0.028559,0.192704,-1.371232,-1.383499,1.247170,1.162131,-0.531932,1.324919,-0.474859,1.405556,-0.013734,1.334753,0.605610
1,False,2010-07-12,-0.153948,-0.720034,-0.277102,-0.895915,-0.837656,-0.909887,-0.544163,0.528628,0.800877,0.781688,-0.416770,-0.642591,-0.531932,-1.270054,0.618431,-1.060554,-0.994312,1.029443,-0.945701
2,False,2015-06-08,-0.060856,0.160431,-0.253060,-1.859565,-0.952400,-1.489897,0.132690,0.240693,1.463794,1.280251,0.010664,-0.098310,-0.531932,1.324919,0.618431,-1.060554,-0.994312,1.029443,1.067425
3,False,2012-04-05,0.373574,-0.594254,0.900990,0.734877,0.883491,0.598137,-3.511904,-2.398706,0.462367,1.137804,0.132788,-0.714207,1.879924,-0.510004,-1.247932,-1.060554,-0.994312,0.033531,-1.362633
4,False,2016-03-05,0.497696,-0.538351,0.179709,-0.747661,-0.149197,-0.213876,1.486396,1.392431,0.561099,0.696220,-0.019867,-0.513682,1.879924,1.324919,-0.474859,0.982439,-0.994312,0.572514,-1.254276


In [18]:
df_desc = df_new.describe()
df_desc.loc["iqr", :] = df_desc.loc["75%", :] -  df_desc.loc["25%", :]
df_desc.loc["iqr_x_1_5", :] = 1.5 * df_desc.loc["iqr", :]
df_desc.loc["lower_whisker", :] = df_desc.loc["25%", :] - df_desc.loc["iqr_x_1_5", :]
df_desc.loc["upper_whisker", :] = df_desc.loc["75%", :] + df_desc.loc["iqr_x_1_5", :]
df_desc

,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,WindGustDir_east,WindGustDir_north,WindDir9am_east,WindDir9am_north,WindDir3pm_east,WindDir3pm_north
count,117590,1.175420e+05,1.175740e+05,1.167950e+05,1.175900e+05,1.175900e+05,1.175900e+05,1.169460e+05,1.163890e+05,1.096330e+05,1.096810e+05,1.175150e+05,1.170050e+05,1.167950e+05,1.175790e+05,1.175790e+05,1.121200e+05,1.121200e+05,1.172320e+05,1.172320e+05
mean,2013-04-16 16:35:27.609490944,-5.316588e-17,8.294520e-16,-1.228902e-17,-1.123913e-17,-5.861268e-17,1.556559e-16,-2.875078e-16,1.225863e-16,1.562537e-14,-3.844028e-14,1.510088e-16,-3.333943e-16,-9.320189e-17,-2.235950e-18,7.856042e-18,4.689633e-17,2.306792e-17,-1.312206e-17,1.309175e-17
min,2007-11-01 00:00:00,-3.210468e+00,-3.934433e+00,-2.771025e-01,-2.526707e+00,-1.640859e+00,-2.185908e+00,-3.563969e+00,-2.446695e+00,-5.024326e+00,-5.428980e+00,-3.698854e+00,-3.893954e+00,-5.319319e-01,-1.376957e+00,-1.356667e+00,-1.483672e+00,-1.400481e+00,-1.374902e+00,-1.362633e+00
25%,2011-01-31 00:00:00,-7.280151e-01,-7.619614e-01,-2.771025e-01,-6.735343e-01,-8.376564e-01,-6.778834e-01,-6.482942e-01,-7.190883e-01,-6.660020e-01,-6.855109e-01,-7.373458e-01,-7.428530e-01,-5.319319e-01,-9.656205e-01,-9.382828e-01,-1.060554e+00,-9.943120e-01,-9.623819e-01,-9.457009e-01
50%,2013-06-16 00:00:00,-2.982513e-02,-7.715478e-02,-2.771025e-01,-8.051901e-02,-1.491974e-01,1.812782e-02,2.855894e-02,7.479418e-04,-3.085355e-03,-1.767708e-03,-5.039818e-02,-6.966317e-02,-5.319319e-01,2.743259e-02,7.178576e-02,-3.905770e-02,-1.373366e-02,3.353052e-02,6.086220e-02
75%,2015-06-21 00:00:00,7.304262e-01,7.194570e-01,-2.049744e-01,5.866232e-01,6.540049e-01,5.981372e-01,7.574777e-01,6.725951e-01,6.739359e-01,6.819755e-01,7.128770e-01,6.894657e-01,-5.319319e-01,1.020486e+00,1.081854e+00,9.824387e-01,9.668447e-01,1.029443e+00,1.067425e+00
max,2017-06-25 00:00:00,3.368033e+00,3.458684e+00,4.391337e+01,7.035664e+00,8.341798e+00,7.906255e+00,1.642593e+00,2.352213e+00,3.297393e+00,3.473927e+00,3.536995e+00,3.568426e+00,1.879924e+00,1.431822e+00,1.500238e+00,1.405556e+00,1.373014e+00,1.441963e+00,1.484357e+00
std,NaN,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
iqr,1602 days 00:00:00,1.458441e+00,1.481418e+00,7.212808e-02,1.260157e+00,1.491661e+00,1.276021e+00,1.405772e+00,1.391683e+00,1.339938e+00,1.367486e+00,1.450223e+00,1.432319e+00,0.000000e+00,1.986106e+00,2.020137e+00,2.042993e+00,1.961157e+00,1.991825e+00,2.013126e+00
iqr_x_1_5,2403 days 00:00:00,2.187662e+00,2.222128e+00,1.081921e-01,1.890236e+00,2.237492e+00,1.914031e+00,2.108658e+00,2.087525e+00,2.009907e+00,2.051230e+00,2.175334e+00,2.148478e+00,0.000000e+00,2.979159e+00,3.030206e+00,3.064489e+00,2.941735e+00,2.987737e+00,3.019689e+00


In [19]:
df_whiskers = df_desc.loc[["lower_whisker", "upper_whisker"],:].T
df_whiskers

,lower_whisker,upper_whisker
Date,2004-07-03 00:00:00,2022-01-18 00:00:00
MinTemp,-2.915677,2.918088
MaxTemp,-2.984089,2.941585
Rainfall,-0.385295,-0.096782
WindGustSpeed,-2.56377,2.476859
WindSpeed9am,-3.075148,2.891497
WindSpeed3pm,-2.591914,2.512168
Humidity9am,-2.756952,2.866135
Humidity3pm,-2.806613,2.76012
Pressure9am,-2.675909,2.683843


In [20]:
df_new = limit_outliers(df_new, df_whiskers=df_whiskers)

In [21]:
df_new

,RainTomorrow,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,WindGustDir_east,WindGustDir_north,WindDir9am_east,WindDir9am_north,WindDir3pm_east,WindDir3pm_north
0,False,2012-01-26,1.211401,1.152702,-0.229017,-0.080519,0.309775,-1.141891,0.028559,0.192704,-1.371232,-1.383499,1.247170,1.162131,-0.531932,1.324919,-0.474859,1.405556,-0.013734,1.334753,0.605610
1,False,2010-07-12,-0.153948,-0.720034,-0.277102,-0.895915,-0.837656,-0.909887,-0.544163,0.528628,0.800877,0.781688,-0.416770,-0.642591,-0.531932,-1.270054,0.618431,-1.060554,-0.994312,1.029443,-0.945701
2,False,2015-06-08,-0.060856,0.160431,-0.253060,-1.859565,-0.952400,-1.489897,0.132690,0.240693,1.463794,1.280251,0.010664,-0.098310,-0.531932,1.324919,0.618431,-1.060554,-0.994312,1.029443,1.067425
3,False,2012-04-05,0.373574,-0.594254,-0.096782,0.734877,0.883491,0.598137,-2.756952,-2.398706,0.462367,1.137804,0.132788,-0.714207,1.879924,-0.510004,-1.247932,-1.060554,-0.994312,0.033531,-1.362633
4,False,2016-03-05,0.497696,-0.538351,-0.096782,-0.747661,-0.149197,-0.213876,1.486396,1.392431,0.561099,0.696220,-0.019867,-0.513682,1.879924,1.324919,-0.474859,0.982439,-0.994312,0.572514,-1.254276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117585,False,2009-01-18,-0.324616,0.887165,-0.277102,-0.525280,-1.181886,-0.677883,-0.283835,-1.198979,0.363634,0.126434,-0.401505,0.718112,-0.531932,1.324919,0.618431,-1.373707,-0.544419,-0.962382,1.067425
117586,False,2016-09-28,-0.929714,-1.460744,-0.096782,1.402019,0.309775,-0.445880,0.340953,0.960530,0.546994,1.123560,-1.042656,-1.602244,1.879924,-0.510004,-1.247932,-0.039058,-1.400481,0.033531,-1.362633
117587,False,2016-11-20,0.901095,0.929092,-0.096782,-0.895915,-0.608170,-0.677883,0.757478,-0.191208,0.546994,0.254636,0.438098,1.004576,1.879924,1.020486,1.081854,-0.039058,1.373014,1.334753,0.605610
117588,False,2012-11-04,-0.417708,1.334385,-0.277102,0.438369,-1.411372,0.134130,0.809543,-1.294957,-0.651897,-1.141340,-0.218319,1.348332,-0.531932,-0.965621,1.081854,1.405556,-0.013734,0.033531,1.484357


We should do the standardization again with the original data and with mean and std calculated anew, but we'll skip this step

In [22]:
df_new = fill_nulls(df_new, df_desc=df_desc)

In [23]:
df_new.isnull().sum()

RainTomorrow         0
Date                 0
MinTemp              0
MaxTemp              0
Rainfall             0
WindGustSpeed        0
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Temp9am              0
Temp3pm              0
RainToday            0
WindGustDir_east     0
WindGustDir_north    0
WindDir9am_east      0
WindDir9am_north     0
WindDir3pm_east      0
WindDir3pm_north     0
dtype: int64

In [24]:
df_new

,RainTomorrow,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,WindGustDir_east,WindGustDir_north,WindDir9am_east,WindDir9am_north,WindDir3pm_east,WindDir3pm_north
0,False,2012-01-26,1.211401,1.152702,-0.229017,-0.080519,0.309775,-1.141891,0.028559,0.192704,-1.371232,-1.383499,1.247170,1.162131,-0.531932,1.324919,-0.474859,1.405556,-0.013734,1.334753,0.605610
1,False,2010-07-12,-0.153948,-0.720034,-0.277102,-0.895915,-0.837656,-0.909887,-0.544163,0.528628,0.800877,0.781688,-0.416770,-0.642591,-0.531932,-1.270054,0.618431,-1.060554,-0.994312,1.029443,-0.945701
2,False,2015-06-08,-0.060856,0.160431,-0.253060,-1.859565,-0.952400,-1.489897,0.132690,0.240693,1.463794,1.280251,0.010664,-0.098310,-0.531932,1.324919,0.618431,-1.060554,-0.994312,1.029443,1.067425
3,False,2012-04-05,0.373574,-0.594254,-0.096782,0.734877,0.883491,0.598137,-2.756952,-2.398706,0.462367,1.137804,0.132788,-0.714207,1.879924,-0.510004,-1.247932,-1.060554,-0.994312,0.033531,-1.362633
4,False,2016-03-05,0.497696,-0.538351,-0.096782,-0.747661,-0.149197,-0.213876,1.486396,1.392431,0.561099,0.696220,-0.019867,-0.513682,1.879924,1.324919,-0.474859,0.982439,-0.994312,0.572514,-1.254276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117585,False,2009-01-18,-0.324616,0.887165,-0.277102,-0.525280,-1.181886,-0.677883,-0.283835,-1.198979,0.363634,0.126434,-0.401505,0.718112,-0.531932,1.324919,0.618431,-1.373707,-0.544419,-0.962382,1.067425
117586,False,2016-09-28,-0.929714,-1.460744,-0.096782,1.402019,0.309775,-0.445880,0.340953,0.960530,0.546994,1.123560,-1.042656,-1.602244,1.879924,-0.510004,-1.247932,-0.039058,-1.400481,0.033531,-1.362633
117587,False,2016-11-20,0.901095,0.929092,-0.096782,-0.895915,-0.608170,-0.677883,0.757478,-0.191208,0.546994,0.254636,0.438098,1.004576,1.879924,1.020486,1.081854,-0.039058,1.373014,1.334753,0.605610
117588,False,2012-11-04,-0.417708,1.334385,-0.277102,0.438369,-1.411372,0.134130,0.809543,-1.294957,-0.651897,-1.141340,-0.218319,1.348332,-0.531932,-0.965621,1.081854,1.405556,-0.013734,0.033531,1.484357
